In [ ]:
import numpy as np
import pandas as pd
import os
import os.path as opath
# import ast
# import json
#from pandas.io.json import json_normalize
import datetime
import pickle
import math

# import matplotlib as plt
# import matplotlib.pyplot as plt
# import seaborn as sb
import plotly 
import plotly.plotly as py
import plotly.presentation_objs as pres
from plotly.graph_objs import *
#import plotly.graph_objs as go
import plotly.offline as offline
from plotly import tools
from IPython.display import Image

from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
#plotly.tools.set_credentials_file(username='saurabhpundir', api_key='evTecysAK8CEVYBe6JUW',proxy_username='spundir', proxy_password='Cowboy@02')
plotly.tools.set_credentials_file(username='saurabhpundir', api_key='evTecysAK8CEVYBe6JUW')

<div class="span5 alert alert-info">
<h2>Note book details</h2>

<p> This notebook is for <b>data story</b> for <b>Mortgage client segmentation</b> project.</p>

<p> Notes.</p>
<ol>
<li>  The file will utlize the pickle file created in third part of for data wrangling</li> 
<li>  The file will perform exploratory analysis on the mortgage data with some </li> 
        
</ol>
</div>

In [ ]:
# from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
cssAnalysis = '.analysis {background:#5bffc0; }'
HTML('<style>{} {}</style>'.format(css,cssAnalysis))
pickle_file='df_selectdata_hmda_cenus.sa'

colorApprove='rgb(50,205,50)'
colorDeny='rgb(255,0,0)'
colorOrange1='rgb(255, 112, 67)'#'rgb(255, 87, 34)'# 1
colorBlue2='rgb(26, 4, 226)' # 2
colorGreen3='rgb(76, 175, 80)'#3
colorYellow4='rgb(255, 235, 59)' #4
colorRed5='rgb(255,0,0)' #5
colorGray='rgb(190,190,190)' # unknown
colorPink='rgb(255,105,180)'

In [ ]:
df_selectdata = pickle.load( open( pickle_file, "rb" ) )
#df_alldata = pickle.load( open( 'df_alldata_hmda_cenus.sa', "rb" ) )
#df_selectdata.info()

In [ ]:
#df_selectdata[df_selectdata.StateName.isnull()]
statelist=df_selectdata.StateName.unique()
#statelist

In [ ]:
### helper methods

In [ ]:
def getCountForAmountRange(df,colname,min,max,grpbycol='ActionType',outputcol='Year'):
    return pd.DataFrame(df[(df[colname]>min) & (df[colname]<=max)].groupby(grpbycol)[outputcol].count())
    

In [ ]:
def getDataFrameGrroupedbyCol(dfselect,colname):
    dfgroup=dfselect.groupby([colname,'Result'], as_index=False)
    dfgraph=dfgroup.count().loc[:,[colname,'Result','Year']]
    dfgraph=dfgraph[dfgraph['Year'].isna()==False]
    dfgraph['Count']=((dfgraph['Year']/1000.0)).apply(math.ceil)
    #dfgraph_col=dfgraph_col.assign(Count=((dfgraph['Year']/1000.0)).apply(math.ceil))
    return dfgraph
    

In [ ]:
def getlistbycolumnForResult(dfselect,colname,value,fulllist=False):
    if ( type(value) == int):
        dfgraph_col=dfselect[dfselect[colname]==value]
    else:
        dfgraph_col=dfselect[dfselect[colname].isin(value)]
    #dfgraph_female=dfgraph[dfgraph.ApplicantSex==2]
    dfgraph_col=dfgraph_col.assign(perc=((dfgraph_col['Count']/dfgraph_col['Count'].sum())*100))
    templist=dfgraph_col.groupby('Result').sum().sort_values('Result').perc.tolist()

    percListtext=list(map((lambda x:str(round(x))+'%'),templist))
    if(fulllist):
        percList=list(map((lambda x:round(x)),templist))
    else:
        percList=list(map((lambda x:round(x/2)),templist))
        percList.insert(0, 50)
        percListtext.insert(0, 0)

    return percList,percListtext


In [ ]:
def getlistofResultbycolumn(dfselect,value,fulllist=True):
    if (type(value) == int):
        dfgraph_col=dfselect[dfselect.Result==value]
    else:
        dfgraph_col=dfselect[dfselect.Result.isin(value)]
    dfgraph_col=dfgraph_col.assign(perc=((dfgraph_col['Count']/dfgraph_col['Count'].sum())*100))
    #templist=dfgraph_col.groupby('Result').sum().sort_values('Result').perc.tolist()
    templist=dfgraph_col.sort_values('Result').perc.tolist()

    percListtext=list(map((lambda x:str(round(x))+'%'),templist))
    if(fulllist):
        percList=list(map((lambda x:round(x)),templist))
    else:
        percList=list(map((lambda x:round(x/2)),templist))
        percList.insert(0, 50)
        percListtext.insert(0, 0)

    return percList,percListtext


In [ ]:
def createofflineChart(fig,filenameval,image=False,displayonpage=False):
    if(image):
        offline.plot(fig, filename=filenameval,image='png',image_filename=filenameval,output_type='file')
        #py.image.save_as(fig, filename=filenameval+'.png')
        #Image(filenameval+'.png')
    else:
        offline.plot(fig, filename=filenameval+'.html')
    if(displayonpage):
        Image(filenameval+'.png')

In [ ]:
def createbasemeterForDonut(domainval,inputPercList,inputPercListText,labellist):
    base_chart = {
        "values": [40, 10, 10, 10, 10, 10, 10],
        "labels": ["-", "0", "20", "40", "60", "80", "100"],
       'domain':domainval,
        "marker": {
            "colors": [
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)',
                'rgb(255, 255, 255)'
            ],
            "line": {
                "width": 1
            }
        },
        "name": "Gauge",
        "hole": .4,
        "type": "pie",
        "direction": "clockwise",
        "rotation": 108,
        "showlegend": False,
        "hoverinfo": "none",
        "textinfo": "text",
        "textposition": "outside"
    }
    meter_chart = {
        "values": inputPercList,
        "labels":labellist,
        "text":inputPercListText,
        "marker": {
            'colors': [
                'rgb(255, 255, 255)',
                colorDeny,
                 colorApprove,
            ]
        },
        "domain": domainval,
        "name": "Gauge",
        "hole": .3,
        "type": "pie",
        "direction": "clockwise",
        "rotation": 90,
        "showlegend": True,
        "textinfo": "text",
        "textposition": "inside",
         "hoverinfo": "none",
    }
    # we don't want the boundary now
    base_chart['marker']['line']['width'] = 0
    
    return base_chart,meter_chart


In [ ]:
def createHalfDonut(title,inputPercList,inputPercListText,labellist):
    base_chart,meter_chart=createbasemeterForDonut({"x": [0, .48]},inputPercList,inputPercListText,labellist)
    layout = {
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'shapes': [
            {
                'type': 'path',
                'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
                'fillcolor': 'rgba(44, 160, 101, 0.5)',
                'line': {
                    'width': 0.5
                },
                'xref': 'paper',
                'yref': 'paper'
            }
        ],
        'annotations': [
            {
                'xref': 'paper',
                'yref': 'paper',
                'x': 0.23,
                'y': 0.45,
                'text': '50',
                'showarrow': False
            }
        ]
    }
    return base_chart, meter_chart,layout 

In [ ]:
def createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labellist,filenameval):
   
    inputPercListText1[0]=labellist[0]
    inputPercListText2[0]=labellist[1]
    inputPercListText3[0]=labellist[2]

    
    labels=["","Denied","Accepted"]
    domain1={'x':[0, 0.33], 'y':[0.66,1.0]}# for the cell (1,1)
    domain2={'x':[0.66, 1], 'y':[0.66,1]}#cell (1,2)
    domain3={'x':[0, 0.33], 'y':[0.33, 0.66]}#cell (2,1)

    base_chart1,meter_chart1=createbasemeterForDonut(domain1,inputPercList1,inputPercListText1,labels)
    base_chart2,meter_chart2=createbasemeterForDonut(domain2,inputPercList2,inputPercListText2,labels)
    base_chart3,meter_chart3=createbasemeterForDonut(domain3,inputPercList3,inputPercListText3,labels)
    
    layout = {
        'autosize':False,
        'height' : 1000,
        'width' : 900,
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        }#,
#         'shapes': [
#             {
#                 'type': 'path',
#                 'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
#                 'fillcolor': 'rgba(44, 160, 101, 0.5)',
#                 'line': {
#                     'width': 0.5
#                 },
#                 'xref': 'paper',
#                 'yref': 'paper'
#             }
#         ],
#         'annotations': [
#             {
#                 'xref': 'paper',
#                 'yref': 'paper',
#                 'x': 0.23,
#                 'y': 0.45,
#                 'text': '50',
#                 'showarrow': False
#             }
#        ]
    }
#     fig = {"data": [base_chart1, base_chart2,base_chart3, base_chart4,
#                     meter_chart1,meter_chart2, meter_chart3, meter_chart4],
    fig = {"data": [
                    meter_chart1,meter_chart2, meter_chart3],
           "layout": layout }
    createofflineChart(fig,filenameval)

In [ ]:
def createFourHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,inputPercList4,inputPercListText4,labellist,filenameval):
    
    labels=["","Denied","Accepted"]
    inputPercListText1[0]=labellist[0]
    inputPercListText2[0]=labellist[1]
    inputPercListText3[0]=labellist[2]
    inputPercListText4[0]=labellist[3]
    
    domain1={'x': [0, .48],'y': [0, .49]}# for the cell (1,1)
    domain2={'x': [.51, 1],'y': [0, .49]}#cell (1,2)
    domain3={'x': [0, .48],'y': [.51, 1]}#cell (2,1)
    domain4={'x': [.51, 1],'y': [.51, 1]}#cell (2,2)

    base_chart1,meter_chart1=createbasemeterForDonut(domain1,inputPercList1,inputPercListText1,labels)
    base_chart2,meter_chart2=createbasemeterForDonut(domain2,inputPercList2,inputPercListText2,labels)
    base_chart3,meter_chart3=createbasemeterForDonut(domain3,inputPercList3,inputPercListText3,labels)
    base_chart4,meter_chart4=createbasemeterForDonut(domain4,inputPercList4,inputPercListText4,labels)

    layout = {
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        }#,
#         'shapes': [
#             {
#                 'type': 'path',
#                 'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
#                 'fillcolor': 'rgba(44, 160, 101, 0.5)',
#                 'line': {
#                     'width': 0.5
#                 },
#                 'xref': 'paper',
#                 'yref': 'paper'
#             }
#         ],
#         'annotations': [
#             {
#                 'xref': 'paper',
#                 'yref': 'paper',
#                 'x': 0.23,
#                 'y': 0.45,
#                 'text': '50',
#                 'showarrow': False
#             }
#        ]
    }
    fig = {"data": [
                    meter_chart1,meter_chart2, meter_chart3, meter_chart4],
           "layout": layout }
    createofflineChart(fig,filenameval)

In [ ]:
def createSixHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,inputPercList4,inputPercListText4,inputPercList5,
                       inputPercListText5,inputPercList6,inputPercListText6,labellist,filenameval):
   
    inputPercListText1[0]=labellist[0]
    inputPercListText2[0]=labellist[1]
    inputPercListText3[0]=labellist[2]
    inputPercListText4[0]=labellist[3]
    inputPercListText5[0]=labellist[4]
    inputPercListText6[0]=labellist[5]
    
    labels=["","Denied","Accepted"]
    domain1={'x':[0, 0.33], 'y':[0.66,1.0]}# for the cell (1,1)
    domain2={'x':[0.66, 1], 'y':[0.66,1]}#cell (1,2)
    domain3={'x':[0, 0.33], 'y':[0.33, 0.66]}#cell (2,1)
    domain4={'x':[0.66, 1], 'y':[0.33, 0.66]}#cell (2,2)
    domain5={'x':[0, 0.33], 'y':[0,0.33]}#cell (3,1)
    domain6={'x': [0.66, 1], 'y':[0,0.33]}#cell (3,1)

#     domain1={'x': [0, .48],'y': [0, .32]}# for the cell (1,1)
#     domain2={'x': [.52, 1],'y': [0, .32]}#cell (1,2)
#     domain3={'x': [0, .48],'y': [.34, .65]}#cell (2,1)
#     domain4={'x': [.52, 1],'y': [.34, .65]}#cell (2,2)
#     domain5={'x': [0, .48],'y': [.67, 1]}#cell (3,1)
#     domain6={'x': [.52, 1],'y': [.67, 1]}#cell (3,1)

    #domain1={'x': [0, .48],'y': [0, .32]}
    base_chart1,meter_chart1=createbasemeterForDonut(domain1,inputPercList1,inputPercListText1,labels)
    #print(inputPercListText1[0])
    #domain2={'x': [.52, 1],'y': [0, .32]}
    base_chart2,meter_chart2=createbasemeterForDonut(domain2,inputPercList2,inputPercListText2,labels)
    #print(inputPercListText2[0])
    #domain3={'x': [0, .48],'y': [.34, .65]}
    base_chart3,meter_chart3=createbasemeterForDonut(domain3,inputPercList3,inputPercListText3,labels)
    #print(inputPercListText3[0])
    #domain4={'x': [.52, 1],'y': [.34, .65]}
    base_chart4,meter_chart4=createbasemeterForDonut(domain4,inputPercList4,inputPercListText4,labels)
    #print(inputPercListText4[0])
    #domain5={'x': [0, .48],'y': [.67, 1]}
    base_chart5,meter_chart5=createbasemeterForDonut(domain5,inputPercList5,inputPercListText5,labels)
    #print(inputPercListText5[0])
    #domain6={'x': [.52, 1],'y': [.67, 1]}
    base_chart6,meter_chart6=createbasemeterForDonut(domain6,inputPercList6,inputPercListText6,labels)
#     print(inputPercListText6[0])

#     fig = tools.make_subplots(rows=3, cols=2, subplot_titles=('Plot 1', 'Plot 2',
#                                                           'Plot 3', 'Plot 4', 'Plot 5', 'Plot 6'))
    
#     fig.append_trace(meter_chart1, 1, 1)
#     fig.append_trace(meter_chart2, 1, 2)
#     fig.append_trace(meter_chart3, 2, 1)
#     fig.append_trace(meter_chart4, 2, 2)
#     fig.append_trace(meter_chart3, 3, 1)
#     fig.append_trace(meter_chart4, 3, 2)
    
    layout = {
        'autosize':False,
        'height' : 1000,
        'width' : 900,
        'title':title,
        'xaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        },
        'yaxis': {
            'showticklabels': False,
            'showgrid': False,
            'zeroline': False,
        }#,
#         'shapes': [
#             {
#                 'type': 'path',
#                 'path': 'M 0.235 0.5 L 0.24 0.65 L 0.245 0.5 Z',
#                 'fillcolor': 'rgba(44, 160, 101, 0.5)',
#                 'line': {
#                     'width': 0.5
#                 },
#                 'xref': 'paper',
#                 'yref': 'paper'
#             }
#         ],
#         'annotations': [
#             {
#                 'xref': 'paper',
#                 'yref': 'paper',
#                 'x': 0.23,
#                 'y': 0.45,
#                 'text': '50',
#                 'showarrow': False
#             }
#        ]
    }
#     fig = {"data": [base_chart1, base_chart2,base_chart3, base_chart4,
#                     meter_chart1,meter_chart2, meter_chart3, meter_chart4],
    fig = {"data": [
                    meter_chart1,meter_chart2, meter_chart3, meter_chart4, meter_chart5, meter_chart6],
           "layout": layout }
    #fig = {"data": [meter_chart1, meter_chart2, meter_chart3, meter_chart4]}
    #py.iplot(fig, filename='gauge-meter-chart-Male-State')
    #offline.plot(fig, filename=filenameval)
    createofflineChart(fig,filenameval)
   
    #return base_chart, meter_chart,layout 

In [ ]:
# graph for 0-1101
# graph for 1101-5101
# graph for 5101-99999

def drawIncomegraphonRange(df,title,start, end):
    #start=0 
    #end=1101    
    dfgraph=df[(np.int64(df.start)>=np.int64(start))& (np.int64(df.end)<=np.int64(end))].sort_values(by=['start'])
    #dfgraph=df_income_range[(np.int64(df_income_range.start)>np.int64(start)]
    #dfgraph=df_income_range[(np.int64(df_income_range.start)>np.int64(start))]                         
    #dfgraph=df_income_range[(np.int64(df_income_range.end)<=np.int64(end))].sort_values(by=['start'])
    
    traceApprove =graph_objs.Bar( 
      x= dfgraph[dfgraph['ActionType']==1].range, 
      y= dfgraph[dfgraph['ActionType']==1].value, 
      name= "Loan originated(Approved)",
          #orientation = 'h',
    marker=dict(
            color='rgb(50,205,50)',
            line=dict(
                color='rgb(34,139,34)',
                width=1.5,
            )
        ),
        opacity=0.6,

    )
    
    traceNotAccepted = graph_objs.Bar(
      x= dfgraph[dfgraph['ActionType']==2].range, 
      y= dfgraph[dfgraph['ActionType']==2].value,
      name= "Application approved but not accepted", 
          #orientation = 'h',
    marker=dict(
            color='rgb(255,255,0)',
            line=dict(
                color='rgb(255,255,0)',
                width=1.5,
            )
        ),
        opacity=0.6
    )
    traceDenied = graph_objs.Bar(
      x= dfgraph[dfgraph['ActionType']==3].range, 
      y= dfgraph[dfgraph['ActionType']==3].value,
      name= "Application denied by financial institution", 
          #orientation = 'h',
    marker=dict(
            color='rgb(255,0,16)',
            line=dict(
                color='rgb(255,0,16)',
                width=1.5,
            )
        ),
        opacity=0.6
    )
    tracePurchased = graph_objs.Bar(
      x= dfgraph[dfgraph['ActionType']==6].range, 
      y= dfgraph[dfgraph['ActionType']==6].value,
      name= "Loan purchased by your institution", 
          #orientation = 'h',
    marker=dict(
            color='rgb(190,190,190)',
            line=dict(
                color='rgb(119,136,153)',
                width=1.5,
            )
        ),
        opacity=0.6
    )
    tracePreapproveDenied = graph_objs.Bar(
      x= dfgraph[dfgraph['ActionType']==7].range, 
      y= dfgraph[dfgraph['ActionType']==7].value,
      name= "Preapproval request denied by financial institution", 
          #orientation = 'h',
    marker=dict(
            color='rgb(121, 0, 0)',
            line=dict(
                color='rgb(121, 0, 0)',
                width=1.5,
            )
        ),
        opacity=0.6
    )
    tracePreapproveNotAccepted = graph_objs.Bar(
      x= dfgraph[dfgraph['ActionType']==8].range, 
      y= dfgraph[dfgraph['ActionType']==8].value,
      name= "Preapproval request approved but not accepted", 
          #orientation = 'h',
    marker=dict(
            color='rgb(135,206,250)',
            line=dict(
                color='rgb(135,206,250)',
                width=1.5,
            )
        ),
        opacity=0.6
    )
    data = Data([traceApprove,traceNotAccepted,traceDenied,tracePurchased, tracePreapproveDenied,tracePreapproveNotAccepted])


    layout = graph_objs.Layout(
        title=title,
        width=1050,
        height=950,
        yaxis=dict(
            tickfont=dict(
                size=10,
                color='rgb(107, 107, 107)'
            )
        ),
        xaxis=dict(
            title='Amount in (000)',
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )

        ),
    #    legend=dict(
    #        x=0,
    #        y=1.0,
    #        bgcolor='rgba(255, 255, 255, 0)',
    #        bordercolor='rgba(255, 255, 255, 0)'
    #    ),
        barmode='group',
        bargap=1.15,
        bargroupgap=1.55
    )
    #print((traceApprove.y.sum()))
    return data,layout,dfgraph 

#fig = Figure(data=data, layout=layout)
#return fig
#py.iplot(fig, filename='grouped-bar-state-salary')

In [ ]:
def createHalfGuagechart(base_chart, meter_chart,layout,filenameval):
    fig = {"data": [base_chart, meter_chart],
       "layout": layout }
    #py.iplot(fig, filename='gauge-meter-chart-Male-State')
    #offline.plot(fig, filename=filenameval)
    createofflineChart(fig,filenameval)

In [ ]:
def createPiChartForThreeValues(valuesfirst,valuessecond,valuethird,labels,title,filename,namefirst,namesecond,namethird):
    fig = {
      "data": [
        {
          "values": valuesfirst,
          "labels":labels, 
          "text":[namefirst],
          "textposition":"inside",
          "domain": {"x": [0, .33]},
          "name": namefirst,#"GHG Emissions",
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        },
        {
          "values": valuessecond,
          "labels": labels,
          "text":[namesecond],
          "textposition":"inside",
          "domain": {"x": [.35, 0.64]},
          "name": namesecond,
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        },
          {
          "values": valuessecondthird,
          "labels": labels,
          "text":[namethird],
          "textposition":"inside",
          'domain': {'x': [0.67, 1]},
          "name": namethird,
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        }
      ],
      "layout": {
            "title":title,
            "annotations": [
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namefirst,
                    "x": 0.15,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namesecond,
                    "x": 0.65,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namethird,
                    "x": 0.9,
                    "y": 0.5
                }
            ]
        }
    }
    return fig
    #return py.iplot(fig, filename='donut')

In [ ]:
def createPiChartFortwoValues(valuesfirst,valuessecond,labels,title,filename,namefirst,namesecond):
    fig = {
      "data": [
        {
          "values": valuesfirst,#[16, 15, 12, 6, 5, 4, 42],
          "labels":labels, #[
#             "US",
#             "China",
#             "European Union",
#             "Russian Federation",
#             "Brazil",
#             "India",
#             "Rest of World"
#           ],
          "text":[namefirst],
          "textposition":"inside",
          "domain": {"x": [0, .48]},
          "name": namefirst,#"GHG Emissions",
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        },
        {
          "values": valuessecond,
          "labels": labels,
          "text":[namesecond],
          "textposition":"inside",
          "domain": {"x": [.52, 1]},
          "name": namesecond,
          "hoverinfo":"label+percent+name",
          "hole": .4,
          "type": "pie"
        }
      ],
      "layout": {
            "title":title,
            "annotations": [
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namefirst,
                    "x": 0.20,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": namesecond,
                    "x": 0.8,
                    "y": 0.5
                }
            ]
        }
    }
    return fig
    #py.iplot(fig, filename='donut')

In [ ]:
def getDataFrameGroupByColForState(dfselect,colname,divisor=1000.0):
    dfgroup=df_selectdata.groupby([colname,'StateName'], as_index=False)
    dfgraph=dfgroup.count().loc[:,[colname,'StateName','Year']]
    dfgraph=dfgraph[dfgraph['Year'].isna()==False]
    dfgraph['Count']=((dfgraph['Year']/divisor)).apply(math.ceil)  
    return dfgraph

In [ ]:
def getbarForColumnForStateGraph(name,yaxis,color):
    trace =graph_objs.Bar( 
      y= yaxis,#dfgraph[dfgraph['ApplicantSex']==2]['Count'], 
      x= statelist, 
      name= name,
    marker=dict(
            color=color,
            line=dict(
                color=color,
                width=1.5,
            )
        ),
        opacity=0.6
    )
    return trace
    

## us map graph

In [ ]:
dfgroup=df_selectdata.groupby('StateName', as_index=False)
dfgraph=dfgroup.count().loc[:,['StateName','Year']]
dfgraph['LoanAmount']=dfgroup.sum()['LoanAmount']#.loc[:,['LoanAmount','Year']]
dfgraph['LoanAmount'] =((dfgraph['LoanAmount']/1000000.0)).apply(math.ceil)

In [ ]:
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

for col in dfgraph.columns:
    dfgraph[col] = dfgraph[col].astype(str)
    
dfgraph['text'] = dfgraph['StateName'] + '<br>' +\
    'LoanAmount '+dfgraph['LoanAmount']+' USD<br>'+\
    'No of Applicant '+dfgraph['Year']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = dfgraph['StateName'],
        z = dfgraph['LoanAmount'].astype(float),
        locationmode = 'USA-states',
        text = dfgraph['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = 'Loan Application by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
#py.iplot( fig, filename='d3-cloropleth-loan' )
createofflineChart(fig,'USMap-cloropleth-loan')

### bar graph for state and action for each field

#### Gender state scatter plot

In [ ]:
def getbargraphforstate(data,title,filename,barmode='group',width=950):
    
    #data = Data([traceFemale, traceMale,traceUnknown])

    layout = graph_objs.Layout(
        title=title,
        width=width,
        xaxis=dict(
            tickfont=dict(
                size=10,
                color='rgb(107, 107, 107)'
            )
        ),
        yaxis=dict(
            title='Counts in(1000)',
            titlefont=dict(
                size=16,
                color='rgb(107, 107, 107)'
            ),
            tickfont=dict(
                size=14,
                color='rgb(107, 107, 107)'
            )
        ),
    #    legend=dict(
    #        x=0,
    #        y=1.0,
    #        bgcolor='rgba(255, 255, 255, 0)',
    #        bordercolor='rgba(255, 255, 255, 0)'
    #    ),
        barmode=barmode,
        bargap=0.15,
        bargroupgap=0.2
    )


    fig = Figure(data=data, layout=layout)
    #py.iplot(fig, filename='grouped-bar-gender')
    createofflineChart(fig,filename)    

#### Gender bar graph

In [ ]:
colname='ApplicantSex'
dfgraph=getDataFrameGroupByColForState(df_selectdata,colname)

In [ ]:
title='Loan counts for gender for all US state'
#labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not provided ','Not applicable']
labels=['Men','Women','Unknown']
filename='bar-chart-gender-State'

traceMale=getbarForColumnForStateGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorBlue2)
traceFemale=getbarForColumnForStateGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorPink)
traceUnknown=getbarForColumnForStateGraph(labels[2],dfgraph[(dfgraph[colname]==3)|(dfgraph[colname]==4)]['Count'],colorGray)

data = Data([traceFemale, traceMale,traceUnknown])
#getbargraphforstate(data,'Loan counts for gender for all US state','Gender-LoanCount-Bar')
getbargraphforstate(data,title,filename)

#### Race bar graph

In [ ]:
colname='ApplicantRace'
dfgraph=getDataFrameGroupByColForState(df_selectdata,colname)

In [ ]:
title='Loan counts for Race for all US state'
labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander',
        'White','Information not provided ','Not applicable']
filename='bar-chart-ApplicantRace-State'

traceAI=getbarForColumnForStateGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorYellow4)
traceAs=getbarForColumnForStateGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceBL=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorGreen3)
traceNH=getbarForColumnForStateGraph(labels[3],dfgraph[dfgraph[colname]==4]['Count'],colorRed5)
traceW=getbarForColumnForStateGraph(labels[4],dfgraph[dfgraph[colname]==5]['Count'],colorOrange1)

traceUnkown=getbarForColumnForStateGraph(labels[5],dfgraph[(dfgraph[colname]==6)|(dfgraph[colname]==7)]['Count'],colorGray)

data = Data([traceAI, traceAs,traceBL,traceNH,traceW,traceUnkown])
getbargraphforstate(data,title,filename,'stack',1250)
#getbargraphforstate(data,title,filename,width=1250)

#### Ethinicity bar graph

In [ ]:
colname='ApplicantEthnicity'
dfgraph=getDataFrameGroupByColForState(df_selectdata,colname)

In [ ]:
title='Loan counts for Ethnicity for all US state'
labels=['Hispanic or Latino','Not Hispanic or Latino','Information not provided ','Not applicable']
filename='bar-chart-Ethnicity-State'

traceHisp=getbarForColumnForStateGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorBlue2)
traceNoHisp=getbarForColumnForStateGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorOrange1)
# traceBL=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],'rgb(0, 188, 212)')
# traceNH=getbarForColumnForStateGraph(labels[3],dfgraph[dfgraph[colname]==4]['Count'],'rgb(255, 87, 34)')
# traceW=getbarForColumnForStateGraph(labels[4],dfgraph[dfgraph[colname]==5]['Count'],'rgb(63, 81, 181)')

traceUnkown=getbarForColumnForStateGraph(labels[3],dfgraph[(dfgraph[colname]==3)]['Count'],colorGray)

data = Data([traceHisp, traceNoHisp,traceUnkown])
getbargraphforstate(data,title,filename)
#getbargraphforstate(data,title,filename,width=1250)

#### Action bar graph

In [ ]:
colname='ActionType'
dfgraph=getDataFrameGroupByColForState(df_selectdata,colname)

In [ ]:
title='Loan counts for Action for all US state'
labels=['Loan originated' ,'Application approved but not accepted' , 'Application denied by financial institution', 
        'Application withdrawn by applicant', 'File closed for incompleteness',' Loan purchased by your institution',
        'Preapproval request denied by financial institution','Preapproval request approved but not accepted']
filename='bar-chart-Action-State'

traceLO=getbarForColumnForStateGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceApp=getbarForColumnForStateGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceDen=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
tracepur=getbarForColumnForStateGraph(labels[5],dfgraph[dfgraph[colname]==6]['Count'],colorYellow4)
tracePreDeny=getbarForColumnForStateGraph(labels[6],dfgraph[dfgraph[colname]==7]['Count'],colorOrange1)
tracePreNtAcc=getbarForColumnForStateGraph(labels[7],dfgraph[dfgraph[colname]==8]['Count'],colorGray)

#traceUnkown=getbarForColumnForStateGraph(labels[3],dfgraph[(dfgraph[colname]==4)|(dfgraph[colname]==5)]['Count'],'rgb(190,190,190)')

data = Data([traceLO, traceApp,traceDen,tracepur,tracePreDeny,tracePreNtAcc])
getbargraphforstate(data,title,filename,barmode='stack')
#getbargraphforstate(data,title,filename,width=1250)

#### Loan Type bar graph # did not have data in select data

#### Loan Purpose bar graph

In [ ]:
colname='LoanPurpose'
dfgraph=getDataFrameGroupByColForState(df_selectdata,colname)

In [ ]:
title='Loan counts for Loan Purpose for all US state'
labels=['Home purchase',  'Home improvement', 'Refinancing']
filename='bar-chart-LoanPurpose-State'

traceFam=getbarForColumnForStateGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceMan=getbarForColumnForStateGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceMulti=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
#traceFSA=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)

data = Data([traceFam, traceMan,traceMulti])
getbargraphforstate(data,title,filename,barmode='group')
#getbargraphforstate(data,title,filename,width=1250)

#### Property Type bar graph

In [ ]:
colname='PropertyType'
dfgraph=getDataFrameGroupByColForState(df_selectdata,colname)

In [ ]:
title='Loan counts for Property Type for all US state'
labels=['One to four-family',  'Manufactured housing', 'Multifamily']
filename='bar-chart-PropertyType-State'

traceFam=getbarForColumnForStateGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceMan=getbarForColumnForStateGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceMulti=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorRed5)
#traceFSA=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)

data = Data([traceFam, traceMan,traceMulti])
getbargraphforstate(data,title,filename,barmode='group')
#getbargraphforstate(data,title,filename,width=1250)

#### Owner Occupancy bar graph

In [ ]:
colname='Occupancy'
dfgraph=getDataFrameGroupByColForState(df_selectdata,colname)

In [ ]:
title='Loan counts for Owner Occupancy for all US state'
labels=['Owner-occupied as a principal dwelling',  'Not owner-occupied', 'Not applicable']
filename='bar-chart-OwnerOccupancy-State'

traceFam=getbarForColumnForStateGraph(labels[0],dfgraph[dfgraph[colname]==1]['Count'],colorGreen3)
traceMan=getbarForColumnForStateGraph(labels[1],dfgraph[dfgraph[colname]==2]['Count'],colorBlue2)
traceMulti=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==3]['Count'],colorGray)
#traceFSA=getbarForColumnForStateGraph(labels[2],dfgraph[dfgraph[colname]==4]['Count'],colorYellow4)

data = Data([traceFam, traceMan,traceMulti])
getbargraphforstate(data,title,filename,barmode='group')
#getbargraphforstate(data,title,filename,width=1250)

### Action and state bar

In [ ]:
dfgroup=df_selectdata.groupby(['ActionType','StateName'], as_index=False)
dfgraph=dfgroup.count().loc[:,['ActionType','StateName','Year']]
#dfgraph=dfgraph[dfgraph['Year'].isna()==False]
dfgraph['Count']=((dfgraph['Year']/1000.0)).apply(math.ceil)

In [ ]:
traceApprove =graph_objs.Bar( 
  x= dfgraph[dfgraph['ActionType']==1]['Count'], 
  y= statelist, 
  name= "Loan originated",
      orientation = 'h',
marker=dict(
        color='rgb(50,205,50)',
        line=dict(
            color='rgb(34,139,34)',
            width=1.5,
        )
    ),
    opacity=0.6,
    
)
traceNotAccepted = graph_objs.Bar(
  x= dfgraph[dfgraph['ActionType']==2]['Count'], 
  y= statelist,
  name= "Application approved but not accepted", 
      orientation = 'h',
marker=dict(
        color='rgb(255,255,0)',
        line=dict(
            color='rgb(255,255,0)',
            width=1.5,
        )
    ),
    opacity=0.6
)
tracePurchased = graph_objs.Bar(
  x= dfgraph[(dfgraph['ActionType']==6)]['Count'], 
  y= statelist,
  name= "Loan purchased by your institution", 
      orientation = 'h',
marker=dict(
        color='rgb(190,190,190)',
        line=dict(
            color='rgb(119,136,153)',
            width=1.5,
        )
    ),
    opacity=0.6
)
traceDenied = graph_objs.Bar(
  x= dfgraph[(dfgraph['ActionType']==7)]['Count'], 
  y= statelist,
  name= "Preapproval request denied by financial institution", 
      orientation = 'h',
marker=dict(
        color='rgb(255,0,0)',
        line=dict(
            color='rgb(255,0,0)',
            width=1.5,
        )
    ),
    opacity=0.6
)
tracePreapproveNotAccepted = graph_objs.Bar(
  x= dfgraph[(dfgraph['ActionType']==8)]['Count'], 
  y= statelist,
  name= "Preapproval request approved but not accepted", 
      orientation = 'h',
marker=dict(
        color='rgb(135,206,250)',
        line=dict(
            color='rgb(135,206,250)',
            width=1.5,
        )
    ),
    opacity=0.6
)
data = Data([traceApprove,traceNotAccepted,tracePurchased, traceDenied,tracePreapproveNotAccepted])

layout = graph_objs.Layout(
    title='Loan counts for status for all US state',
    width=950,
    height=950,
    yaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)'
        )
    ),
    xaxis=dict(
        title='Counts (000)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
#    legend=dict(
#        x=0,
#        y=1.0,
#        bgcolor='rgba(255, 255, 255, 0)',
#        bordercolor='rgba(255, 255, 255, 0)'
#    ),
    barmode='stack',
    bargap=0.15,
    bargroupgap=0.2
)


fig = Figure(data=data, layout=layout)
#py.iplot(fig, filename='grouped-bar-state')
createofflineChart(fig,'grouped-bar-state-gender')

### Histogram graph

#### Income & action bar

In [ ]:
# The logic to conver the income by range with action type
count=-1
rangestart=0
rangeend=50
gap=50;
i=0
df_income_range=  pd.DataFrame(columns=['range', 'start', 'end','ActionType','value'])
while(rangeend<10000):
   # df=  pd.DataFrame()
    #df['range']=str.format("{}-{}",rangestart,rangeend)
   # df['start']=rangestart
    #df['end']=rangeend    
    dfcount=getCountForAmountRange(df_selectdata,'ApplicantIncome',rangestart,rangeend)
    for index, row in dfcount.iterrows():
        for j in pd.Series([1,2,3,4,5,6,7,8,9]):
            actiontype=j
            if j==index:
                count=row[0]
            else:
                count=0
            df_income_range.loc[i] =((str.format("{}-{}",rangestart,rangeend)),rangestart,rangeend,actiontype,count)
            i=i+1
            #print(i)
        #df['value']=count
    #print(str.format("{}-{}-{}",rangestart,rangeend,count))
    if(rangeend>=500):
        gap=5000
    rangestart=rangeend
    rangeend=rangeend+gap
        #df_income_range=df_income_range.append(df, ignore_index=True)
        #i=i+1


#df_income_range=df_income_range.append(df, ignore_index=True)
#df_income_range.loc[(len(df_income_range))] =(str.format("{}-{}",rangeend,999999),rangeend,999999,y[(y.ApplicantIncome>rangeend)].Year.count())
i=len(df_income_range)
print('Now '+str(i))
dfcount=getCountForAmountRange(df_selectdata,'ApplicantIncome',rangeend,999999)
for index, row in dfcount.iterrows():
    
    for j in pd.Series([1,2,3,4,5,6,7,8,9]):
            actiontype=j
            if j==index:
                count=row[0]
            else:
                count=0
            df_income_range.loc[i] =((str.format("{}-{}",rangestart,999999)),rangestart,999999,actiontype,count)
            i=i+1

#df_income_m_30=y[(y.ApplicantIncome>rangeend)].Year.count()
#print(df_income_m_30)   
df_income_range=df_income_range.groupby(['start','end','ActionType'],as_index=False).sum()    
df_income_range['range']=df_income_range['start'].astype(str) + '-'+df_income_range['end'].astype(str) 

In [ ]:
# graph for 0-1101
# graph for 1101-5101
# graph for 5101-99999

data,layout,dfgraph=drawIncomegraphonRange(df_income_range,'Loan status grouped by Salary range',0, 999999)
fig = Figure(data=data, layout=layout)
#py.iplot(fig, filename='grouped-bar-state-salary')
createofflineChart(fig,'grouped-bar-state-salary')

#### loan amount state graph

In [ ]:
# The logoc to conver the income by range with action type
count=-1
rangestart=0
rangeend=50
gap=50;
i=0
df_loanamt_range=  pd.DataFrame(columns=['range', 'start', 'end','ActionType','value'])
while(rangeend<10000):
   # df=  pd.DataFrame()
    #df['range']=str.format("{}-{}",rangestart,rangeend)
   # df['start']=rangestart
    #df['end']=rangeend    
    dfcount=getCountForAmountRange(df_selectdata,'LoanAmount',rangestart,rangeend)
    for index, row in dfcount.iterrows():
        for j in pd.Series([1,2,3,4,5,6,7,8,9]):
            actiontype=j
            if j==index:
                count=row[0]
            else:
                count=0
            df_loanamt_range.loc[i] =((str.format("{}-{}",rangestart,rangeend)),rangestart,rangeend,actiontype,count)
            i=i+1
            #print(i)
        #df['value']=count
    #print(str.format("{}-{}-{}",rangestart,rangeend,count))
    if(rangeend>=500):
        gap=5000
    rangestart=rangeend
    rangeend=rangeend+gap
        #df_income_range=df_income_range.append(df, ignore_index=True)
        #i=i+1


#df_income_range=df_income_range.append(df, ignore_index=True)
#df_income_range.loc[(len(df_income_range))] =(str.format("{}-{}",rangeend,999999),rangeend,999999,y[(y.ApplicantIncome>rangeend)].Year.count())
i=len(df_loanamt_range)
print('Now '+str(i))
dfcount=getCountForAmountRange(df_selectdata,'LoanAmount',rangeend,999999)
for index, row in dfcount.iterrows():
    
    for j in pd.Series([1,2,3,4,5,6,7,8,9]):
            actiontype=j
            if j==index:
                count=row[0]
            else:
                count=0
            df_income_range.loc[i] =((str.format("{}-{}",rangestart,999999)),rangestart,999999,actiontype,count)
            i=i+1

#df_income_m_30=y[(y.ApplicantIncome>rangeend)].Year.count()
#print(df_income_m_30)   
df_loanamt_range=df_loanamt_range.groupby(['start','end','ActionType'],as_index=False).sum()    
df_loanamt_range['range']=df_loanamt_range['start'].astype(str) + '-'+df_loanamt_range['end'].astype(str) 

In [ ]:

data,layout,dfgraph=drawIncomegraphonRange(df_loanamt_range,'Loan status grouped by loan amount range',0, 999999)
fig = Figure(data=data, layout=layout)
#py.iplot(fig, filename='grouped-bar-state-loanamt')
createofflineChart(fig,'grouped-bar-state-loanamt')

### Half donut graph for Approve Deny for each field

#### gender and state

In [ ]:
colname='ApplicantRace'
dfselectRace=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
### working on making 1 file 4 graph
#race=1
#raceName='American Indian or Alaska Native'
title='Approved and Deny by Applicant Race'
#labellist=[raceName,"Denied","Accepted"]
labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not provided/NA','Not applicable']
filename='gauge-meter-chart-Race-State.html'
inputPercList1,inputPercListText1=getlistbycolumnForResult(dfselectRace,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForResult(dfselectRace,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForResult(dfselectRace,colname,3)
inputPercList4,inputPercListText4=getlistbycolumnForResult(dfselectRace,colname,4)
inputPercList5,inputPercListText5=getlistbycolumnForResult(dfselectRace,colname,5)
inputPercList6,inputPercListText6=getlistbycolumnForResult(dfselectRace,colname,[6,7])

createSixHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,inputPercList4,inputPercListText4,inputPercList5,inputPercListText5,
                   inputPercList6,inputPercListText6,labels,filename)

#### Gender graph

In [ ]:
colname='ApplicantSex'
dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
title='Approved and Deny by Applicant Gender'
labels=['Men','Women','Unknown']
filename='gauge-meter-chart-gender-State'
inputPercList1,inputPercListText1=getlistbycolumnForResult(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForResult(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForResult(dfselect,colname,[3,4])

createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Ethinicity bar graph

In [ ]:
colname='ApplicantEthnicity'
dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
title='Approved and Deny by Applicant Ethinicity'
labels=['Hispanic or Latino','Not Hispanic or Latino','Information not provided ','Not applicable']
filename='gauge-meter-Ethnicity-State'
inputPercList1,inputPercListText1=getlistbycolumnForResult(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForResult(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForResult(dfselect,colname,[3,4])

createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Loan Type bar graph # we dont have data

df_selectdata.info()

#### Property Type bar graph

In [ ]:
colname='PropertyType'
dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
title='Approved and Deny by Applicant Properrt Type'
labels=['One to four-family',  'Manufactured housing', 'Multifamily']
filename='gauge-meter-PropertyType-State'
inputPercList1,inputPercListText1=getlistbycolumnForResult(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForResult(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForResult(dfselect,colname,3)


createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Owner Occupancy bar graph

In [ ]:
colname='Occupancy'
dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
title='Approved and Deny by Occupancy'
labels=['Owner-occupied as a principal dwelling',  'Not owner-occupied', 'Not applicable']
filename='gauge-meter-OwnerOccupancy-State'
inputPercList1,inputPercListText1=getlistbycolumnForResult(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForResult(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForResult(dfselect,colname,3)


createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

#### Loan Purpose bar graph

In [ ]:
colname='LoanPurpose'
dfselect=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
title='Approved and Deny by Loan Purpose'
labels=['Home purchase',  'Home improvement', 'Refinancing']
filename='gauge-meter-LoanPurpose-State'
inputPercList1,inputPercListText1=getlistbycolumnForResult(dfselect,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForResult(dfselect,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForResult(dfselect,colname,3)


createThreeHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,labels,filename)

### Race

In [ ]:
# get the gender(Applicant Sex) column dataframe
colname='ApplicantRace'
dfselectRace=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
### working on making 1 file 4 graph
#race=1
#raceName='American Indian or Alaska Native'
title='Approved and Deny by Applicant Race'
#labellist=[raceName,"Denied","Accepted"]
labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not provided ','Not applicable']
filename='gauge-meter-chart-Race-State.html'
inputPercList1,inputPercListText1=getlistbycolumnForResult(dfselectRace,colname,1)
inputPercList2,inputPercListText2=getlistbycolumnForResult(dfselectRace,colname,2)
inputPercList3,inputPercListText3=getlistbycolumnForResult(dfselectRace,colname,3)
inputPercList4,inputPercListText4=getlistbycolumnForResult(dfselectRace,colname,4)
inputPercList5,inputPercListText5=getlistbycolumnForResult(dfselectRace,colname,5)
inputPercList6,inputPercListText6=getlistbycolumnForResult(dfselectRace,colname,6)

createSixHalfDonut(title,inputPercList1,inputPercListText1,inputPercList2,inputPercListText2,
                        inputPercList3,inputPercListText3,inputPercList4,inputPercListText4,inputPercList5,inputPercListText5,
                   inputPercList6,inputPercListText6,labels,filename)

### Ethincity

In [ ]:
# get the gender(Applicant Sex) column dataframe
colname='ApplicantEthnicity'
dfselectEhin=getDataFrameGrroupedbyCol(df_selectdata,colname)

### Donut graph for Approve Deny for each field

#### Gender bar graph

In [ ]:
#gender
colname='ApplicantSex'
dfselectProp=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
inputPercList1,inputPercListText1=getlistofResultbycolumn(dfselectProp,0)
inputPercList2,inputPercListText2=getlistofResultbycolumn(dfselectProp,1)
namefirst='Approved'
namesecond='Denied'
title='Distribution by Applicant gender'
labels=['Male','Female','Information not provided ','Not applicable']
filename='pie-chart-gender'
fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
createofflineChart(fig,filename)

#### Race bar graph

In [ ]:
# owner ApplicantRace
colname='ApplicantRace'
dfselectProp=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
inputPercList1,inputPercListText1=getlistofResultbycolumn(dfselectProp,0)
inputPercList2,inputPercListText2=getlistofResultbycolumn(dfselectProp,1)
namefirst='Approved'
namesecond='Denied'
title='Distribution by Applicant Race'
labels=['American Indian or Alaska Native','Asian','Black or African American','Native Hawaiian or Other Pacific Islander','White','Information not provided ','Not applicable']
filename='pie-chart-ApplicantRace'
fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
createofflineChart(fig,filename)

#### Ethinicity bar graph

In [ ]:
# owner ethincity
colname='ApplicantEthnicity'
dfselectProp=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
inputPercList1,inputPercListText1=getlistofResultbycolumn(dfselectProp,0)
inputPercList2,inputPercListText2=getlistofResultbycolumn(dfselectProp,1)
namefirst='Approved'
namesecond='Denied'
title='Distribution by Applicant Ethnicity'
labels=['Hispanic or Latino','Not Hispanic or Latino','Information not provided ','Not applicable']
filename='pie-chart-Ethin'
fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
createofflineChart(fig,filename)

#### Property Type bar graph

In [ ]:
# property type
colname='PropertyType'
dfselectProp=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
# res=1
# propname='One to four-family'
inputPercList1,inputPercListText1=getlistofResultbycolumn(dfselectProp,0)
inputPercList2,inputPercListText2=getlistofResultbycolumn(dfselectProp,1)
namefirst='Approved'
namesecond='Denied'
title='Distribution by Loan Property'
labels=['One to four-family (other than manufactured housing)','Manufactured housing','Multifamily']
filename='pie-chart-propType'
fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
createofflineChart(fig,filename)

#### Owner Occupancy bar graph

In [ ]:
# owner occupancy
colname='Occupancy'
dfselectProp=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
inputPercList1,inputPercListText1=getlistofResultbycolumn(dfselectProp,0)
inputPercList2,inputPercListText2=getlistofResultbycolumn(dfselectProp,1)
namefirst='Approved'
namesecond='Denied'
title='Distribution by Owner Occupancy'
labels=['Owner-occupied as a principal dwelling','Not owner-occupied','Not applicable']
filename='pie-chart-OwnerOccupancy'
fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
createofflineChart(fig,filename)

#### Loan Purpose bar graph

In [ ]:
# loan purpose
colname='LoanPurpose'
dfselectProp=getDataFrameGrroupedbyCol(df_selectdata,colname)

In [ ]:
inputPercList1,inputPercListText1=getlistofResultbycolumn(dfselectProp,0)
inputPercList2,inputPercListText2=getlistofResultbycolumn(dfselectProp,1)
namefirst='Approved'
namesecond='Denied'
title='Distribution by Loan Purpose'
labels=['Home purchase','Home improvement','Refinancing']
filename='pie-chart-LoanPurp'
fig=createPiChartFortwoValues(inputPercList1,inputPercList2,labels,title,filename,namefirst,namesecond)
createofflineChart(fig,filename)

In [ ]:
def url_to_iframe(url, text=True):
    html = ''
    # style
    html += '''<head>
    <style>
    div.textbox {
        margin: 30px;
        font-weight: bold;   
    }
    </style>
    </head>'
    '''
    # iframe
    html += '<iframe src=' + url + '.embed#{} width=750 height=400 frameBorder="0"></iframe>'
    if text:
        html += '''<body>
        <div class="textbox">
            <p>Click on the presentation above and use left/right arrow keys to flip through the slides.</p>
        </div>
        </body>
        '''
    return html

In [ ]:
filename = 'simple-pres'
markdown_string = """
# slide 1
There is only one slide.

---
# slide 2
Again, another slide on this page.

"""

my_pres = pres.Presentation(markdown_string)
pres_url_0 = ' https://plot.ly/~saurabhpundir/0'

#### Gender bar graph

#### Race bar graph

#### Ethinicity bar graph

#### Action bar graph

#### Property Type bar graph

#### Owner Occupancy bar graph

#### Loan Purpose bar graph

#### PurchaserType  bar graph